## Попытка исправить падение показателей FAN алгоритма начиная с 22 релиза

задача #281

In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
from app.cache import LocalCache
from bot import TestAlgorithm
from bot.vis import TaskProgress
from app.config import RunConfig

from concurrent.futures import ThreadPoolExecutor, as_completed

TICKER = 'RNFT'

test_configs = [
    (RunConfig(
        ticker=TICKER,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=1,

        majority_trade=True,
        pretest_period=pretest_period,
        pretest_type=pretest_type,

        threshold_buy_steps=threshold_b_n,
        threshold_sell_steps=threshold_s_n,
        stop_up_p=stop_up_p,
        stop_down_p=stop_down_p,
        # start_time='07:01',
        # step_size_shift='0.02',


        step_size=step_size,
        step_set_orders_cnt=step_cnt,
    ))
    for max_shares in [3]
    for base_shares in [0]
    for threshold_b_n in [0]
    for threshold_s_n in [0]
    for step_size in [1.2]
    for step_cnt in [2]
    for pretest_period in [7]
    for pretest_type in [RunConfig.PRETEST_FAN]
    for stop_up_p in [0]
    for stop_down_p in [0]
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config)
    return test_alg.test(
        last_test_date='2024-12-31',
        test_days_num=365,  # 365,
        shares_count=0,
    )

results = []
LocalCache.clear()
progress = TaskProgress(len(test_configs))

with ThreadPoolExecutor(max_workers=1) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in test_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: float(x['profit_p']), reverse=True)

print()
for item in sorted_results:
    print(item)

Запуск в 01:58
Закончено в 01:58, количество 1, длительность 0:00:22                                               

{'exp': 'RNFT fan ', 'profit': -26451, 'profit_p': -26.45, 'profit_p_avg': -0.07, 'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 4/fan7:0/2 x l76 x 1.2¤ , 'op': 4308}


### FAN

##### v22
    30 - {'profit': -50.65, 'profit_p': -6.64, 'profit_p_avg': -0.22, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'op': 163}
    250 - {'profit': 20.7, 'profit_p': 2.58, 'profit_p_avg': 0.01, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'op': 2266}
    365 -  {'profit': 60.45, 'profit_p': 6.52, 'profit_p_avg': 0.02, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'op': 3239}

826a7e6e тут TradeShiftStrategy начал включаться по FAN флагу. из-за этого произошло падение. до этого на таком кофиге отрабатывал алгоритм как для PRE

##### v23 - v26
    30 - {'profit': -24.3, 'profit_p': -3.18, 'profit_p_avg': -0.11, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'op': 168}
    250 - {'profit': -14.6, 'profit_p': -1.82, 'profit_p_avg': -0.01, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'op': 2583}
    365 - {'profit': 14.7, 'profit_p': 1.58, 'profit_p_avg': 0.0, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'op': 3723}

60b9f70d #251 fan добавлено сужение гэпа. теперь заявки в одну сторону не ставятся через 1 шаг, а на каждый

это уникальная правка для FAN была

    {'profit': -99.0, 'profit_p': -10.67, 'profit_p_avg': -0.03, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'op': 4125}

bd6b0f70 #252 переделал систему расчета прибыли (баг с ростом базового числа и оборота при этом)

Это как для PRE, тот же фикс был     

##### v27
    30 - {'exp': 'RNFT fan ', 'profit': -4419, 'profit_p': -4.42, 'profit_p_avg': -0.15, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l148 x 1.2¤ , 'op': 216}
    250 - {'exp': 'RNFT fan ', 'profit': -6617, 'profit_p': -6.62, 'profit_p_avg': -0.03, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l145 x 1.2¤ , 'op': 2907}
    365 - {'exp': 'RNFT fan ', 'profit': -5175, 'profit_p': -5.17, 'profit_p_avg': -0.01, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l147 x 1.2¤ , 'op': 4125}
    
f81fe495 - #148 расширение временного диапазона торгов до полного дня (как для PRE)

    {'exp': 'RNFT fan ', 'profit': -22254, 'profit_p': -22.25, 'profit_p_avg': -0.06, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l120 x 1.2¤ , 'op': 4913}

3fb4d090 - #148 временно заблокировано срабатывание проверки на выходные (как для PRE)

    {'exp': 'RNFT fan ', 'profit': -32378, 'profit_p': -32.38, 'profit_p_avg': -0.09, 
    'config': RNFT+ 3/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 3/fan7:0/2 x l93 x 1.2¤ , 'op': 3835}
    
c964ddae #148 увеличил минимальное число шагов до 4 в конфиге

    {'exp': 'RNFT fan ', 'profit': -26473, 'profit_p': -26.47, 'profit_p_avg': -0.07, 
    'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 4/fan7:0/2 x l76 x 1.2¤ , 'op': 4308}

824b3f1e #148 переименовал некоторые методы, связанные со временем и рабочими днями, так, чтобы они назывались в соответствии с логикой, добавил расписание торгов для тестовой системы 
- падение произошло из-за введения метода is_weekend()

##### v28 - v29 WIP
    30 - {'exp': 'RNFT fan ', 'profit': -8402, 'profit_p': -8.4, 'profit_p_avg': -0.28, 
    'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 4/fan7:0/2 x l95 x 1.2¤ , 'op': 200}
    250 - {'exp': 'RNFT fan ', 'profit': -17600, 'profit_p': -17.6, 'profit_p_avg': -0.07, 
    'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 4/fan7:0/2 x l85 x 1.2¤ , 'op': 2819}
    365 - {'exp': 'RNFT fan ', 'profit': -32591, 'profit_p': -32.59, 'profit_p_avg': -0.09, 
    'config': RNFT+ 4/fan7:0/2 x l1 x 1.2¤ , 'last_conf': RNFT+ 4/fan7:0/2 x l69 x 1.2¤ , 'op': 4114}
